In [1]:
from cognite.client.exceptions import CogniteException

from tests.auth_utils import get_toolkit_client

In [2]:
from cognite.client.data_classes import FileMetadataWrite
from cognite.client.data_classes.data_modeling import SpaceApply
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteFileApply

In [3]:
client = get_toolkit_client("dev-cluster.env", enable_set_pending_ids=True)

Found dev-cluster.env file in repository root. Loaded variables from dev-cluster.env file.


In [4]:
SPACE = "sp_upgrade"

In [5]:
client.data_modeling.spaces.apply(SpaceApply(space=SPACE))

,value
space,sp_upgrade
is_global,False
last_updated_time,2025-07-04 09:38:40.297000
created_time,2025-07-04 09:38:40.297000


# Happy-Path

In [6]:
file = FileMetadataWrite(
    external_id="file_case1_happy_path",
    name="File Case 1 Happy Path",
    mime_type="text/plain",
)

In [7]:
# Ensure the file does not exist before running the test
client.files.delete(external_id=file.external_id, ignore_unknown_ids=True)

In [8]:
created, upload_link = client.files.create(file)
created

,value
external_id,file_case1_happy_path
name,File Case 1 Happy Path
mime_type,text/plain
labels,[]
id,2805016767704389
created_time,2025-07-04 09:49:28.706000
last_updated_time,2025-07-04 09:49:28.706000
uploaded,False


In [9]:
filecontent = b"This is filecontent"

In [10]:
client.files.upload_content_bytes(filecontent, external_id=created.external_id)

,value
external_id,file_case1_happy_path
name,File Case 1 Happy Path
mime_type,text/plain
id,2805016767704389
created_time,2025-07-04 09:49:28.706000
last_updated_time,2025-07-04 09:49:28.706000
uploaded,False


In [11]:
try:
    updated = client.files.set_pending_ids((SPACE, file.external_id), external_id=file.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    updated

Error 400 Found empty patches for following ids: 


In [13]:
cognite_file = CogniteFileApply(
    space=SPACE,
    external_id=file.external_id,
    name="Case 1 Happy Path",
)

In [14]:
created_dm = client.data_modeling.instances.apply(cognite_file).nodes
created_dm

,space,instance_type,external_id,version,was_modified,last_updated_time,created_time
0,sp_upgrade,node,ts_case1_happy_path,1,True,2025-06-03 05:36:43.807,2025-06-03 05:36:43.807


In [16]:
retrieved_content = client.files.download_bytes(instance_id=cognite_file.as_id())

In [17]:
retrieved_content

,"NodeId(sp_upgrade, ts_case1_happy_path)"
2019-12-31 23:00:00,1.0
2020-01-01 23:00:00,2.0
2020-01-02 23:00:00,3.0


In [18]:
newcontent = b"This is new filecontent"

In [19]:
client.files.upload_content_bytes(newcontent, instance_id=cognite_file.as_id())

In [20]:
retrieved_updatedcontent = client.files.download_bytes(external_id=file.external_id)

In [21]:
retrieved_updatedcontent

,"NodeId(sp_upgrade, ts_case1_happy_path)"
2019-12-31 23:00:00,1.0
2020-01-01 23:00:00,2.0
2020-01-02 23:00:00,3.0
2020-01-03 23:00:00,4.0
2020-01-04 23:00:00,5.0


In [22]:
retrieved_file = client.files.retrieve(external_id=file.external_id)
retrieved_file

,value
external_id,ts_case1_happy_path
instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."
name,Case 1 Happy Path
is_string,False
metadata,{}
is_step,False
security_categories,[]
id,4730781892795296
created_time,2025-06-03 05:30:16.245000
last_updated_time,2025-06-03 05:37:47.732000
